# DSECS: Dipolar Spherical Elementary Current Systems (WIP)

In [ ]:
import logging
import datetime as dt
from swarmpal.io import create_paldata, PalDataItem
from swarmpal.toolboxes import dsecs
import matplotlib.pyplot as plt

In [ ]:
# Enable logging in notebook. Remove this line to disable.
logging.basicConfig(level=logging.INFO, force=True)

## Fetching inputs to the toolbox

In [ ]:
def data_params(sc="A"):
    return dict(
        collection=f"SW_OPER_MAG{sc}_LR_1B",
        measurements=["B_NEC"],
        models=["Model = CHAOS"],  # currently must use name "Model"
        auxiliaries=["QDLat"],
        start_time="2016-03-18T11:00:00",
        end_time="2016-03-18T11:50:00",
        server_url="https://vires.services/ows",
        options=dict(asynchronous=False, show_progress=False),
    )


data = create_paldata(
    PalDataItem.from_vires(**data_params("A")),
    PalDataItem.from_vires(**data_params("C")),
)

print(data)

## Applying the DSECS process

In [ ]:
%%time
process = dsecs.processes.DSECS_Process()
process.set_config(
    dataset_alpha="SW_OPER_MAGA_LR_1B", dataset_charlie="SW_OPER_MAGC_LR_1B"
)
data = process(data)

In [ ]:
print(data)

## Plotting the outputs and fitted magnetic field 

### Simple line plots of the currents from the central latitudinal slice

The outputs in the datatree are enumerated, for each analyzed equatorial crossing included inside the timestamps. (output_0,output_1 etc.) 

In [ ]:
print(data["output_0"].coords)
print(data["output_0"].data_vars)

In [ ]:
latitudes = data["output_0"]["Latitude"][:, 3]
fig, ax = plt.subplots(4, 1, figsize=(10, 20))
lineE = ax[0].plot(
    latitudes, data["output_0"]["JEastTotal"][:, 3], "r", label="Eastward"
)
lineN = ax[0].plot(
    latitudes, data["output_0"]["JNorthTotal"][:, 3], "k", label="Northward"
)
ax[0].set_title("Total Current")
ax[0].set_ylabel("Current density (A/km)")


lineE = ax[1].plot(latitudes, data["output_0"]["JEastCf"][:, 3], "r", label="Eastward")
lineN = ax[1].plot(
    latitudes, data["output_0"]["JNorthCf"][:, 3], "k", label="Northward"
)
ax[1].set_title("Curl free current")
ax[1].set_ylabel("Current density (A/km)")

# ax[0].set_title('Total Current (DF + CF)')

lineE = ax[2].plot(latitudes, data["output_0"]["JEastDf"][:, 3], "r", label="Eastward")
lineN = ax[2].plot(
    latitudes, data["output_0"]["JNorthDf"][:, 3], "k", label="Northward"
)
ax[2].set_title("Divergence free current")
ax[2].set_ylabel("Current density (A/km)")

liner = ax[3].plot(
    latitudes, data["output_0"]["Jr"][:, 3], "r", label="Radial current density"
)
ax[3].set_title("Radial current density")
ax[3].set_ylabel("Current density (nA/m^2)")

for axv in ax:
    axv.set_xlabel("Latitude", loc="left")
    axv.legend()